In [1]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math

from collections.abc import Callable

In [2]:
def random_SSM(N : int) -> (torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor):
    A = torch.autograd.Variable(torch.rand(size=(N,N)), requires_grad = True)
    B = torch.autograd.Variable(torch.rand(size=(N,1)), requires_grad = True)
    C = torch.autograd.Variable(torch.rand(size=(1,N)), requires_grad = True)
    D = torch.autograd.Variable(torch.rand(size=(1,1)), requires_grad = True)
    return A, B, C, D

In [3]:
A, B, C, D = random_SSM(10)


In [4]:
D = torch.zeros((1,1))

In [5]:
delta = torch.tensor(0.01)

In [6]:
def discretize(
    A : torch.Tensor, B : torch.Tensor, C : torch.Tensor, D : torch.Tensor, delta : torch.Tensor
) -> (torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor):
    """Discretizes SSM using bilinear model

    parameters:
        A: (NxN) transition matrix in latent
        B: (Nx1) projection matrix to latent
        C: (1xN) projection matrix from latent to output
        D: (1x1) skip connection from input to output
        delta: time step, ensure sufficient smallness
    """
    Cbar = C
    Dbar = D
    N = A.shape[0]
    Bl = torch.linalg.inv(torch.eye(N) - delta / 2 * A)
    Abar = Bl@(torch.eye(N) + delta/2 * A)
    Bbar = Bl@(delta*B)
    return Abar, Bbar, Cbar, Dbar

In [7]:
Abar, Bbar, Cbar, Dbar = discretize(A, B, C, D, delta)

In [8]:
T = 100
num_steps = int(T/delta)

u = torch.cos(torch.arange(num_steps))

In [9]:
def scan_SSM(
    Ab : torch.Tensor, Bb : torch.Tensor, Cb : torch.Tensor, Db : torch.Tensor,  u : torch.Tensor, x0 : torch.Tensor
) -> torch.Tensor:
    """
    computes steps of the SSM going forward.

    parameters:
        Ab : (NxN) transition matrix in discrete space of latent to latent
        Bb : (Nx1) projcetion matrix from input to latent space
        Cb : (1xN) projection matrix from latent to output
        Db : (1x1) skip connection input to output
        u  : (L,)  trajectory we are trying to track
        x0 : (Nx1) initial condition of latent
    """
    x0 = torch.zeros((10,1))
    x = torch.zeros((Ab.shape[0], len(u[:100])))
    y = torch.zeros_like(u[:100])
    for i in range(u[:100].shape[0]):
        x[:,i] = (Ab@x0 + Bb*u[i]).squeeze()
        y[i] = (Cb@x[:,i]).squeeze()
        x0 = x[:,i].unsqueeze(-1)
    return x, y

In [10]:
def K_conv(Ab : torch.Tensor, Bb : torch.Tensor, Cb : torch.Tensor, L : int) -> torch.Tensor:
    """
    computes convolution window given L time steps using equation K_t = Cb @ (Ab^t) @ Bb. 
    Needs to be flipped for correct causal convolution, but can be used as is in fft mode

    parameters:
        Ab : transition matrix
        Bb : projection matrix from input to latent
        Cb : projection matrix from latent to input
        Db : skip connection
        L  : length over which we want convolutional window
    """
    return torch.stack([(Cb @ torch.matrix_power(Ab, l) @ Bb).squeeze() for l in range(L)])

In [11]:
def causal_conv(u : torch.Tensor, K : torch.Tensor, notfft : bool = False) -> torch.Tensor:
    """
    computes 1-d causal convolution either using standard method or fft transform.

    parameters:
        u : trajectory to convolve
        K : convolutional filter
        notfft: boolean, for whether or not we use fft mode or not.
    """
    assert len(u.shape)==1
    assert K.shape==u.shape
    
    L = u.shape[0]
    powers_of_2 = 2**int(math.ceil(math.log2(2*L)))

    if notfft:
        padded_u = torch.nn.functional.pad(u, (L-1,L-1))
        convolve = torch.zeros_like(u)
        for i in range(L):
            convolve[i] = torch.sum(padded_u[i:i+L]*K.flip(dims=[0]))
        return convolve
    else:

        K_pad = torch.nn.functional.pad(K, (0, L))
        u_pad = torch.nn.functional.pad(u, (0, L))
        
        K_f, u_f = torch.fft.rfft(K_pad, n = powers_of_2), torch.fft.rfft(u_pad, n = powers_of_2)
        return torch.fft.irfft(K_f * u_f, n = powers_of_2)[:L]

In [12]:
K = K_conv(Abar, Bbar, Cbar, 100)

In [13]:
conv_fft = causal_conv(u[:100], K)
conv_notfft = causal_conv(u[:100],K , notfft=True)

In [14]:
x, y = scan_SSM(Abar, Bbar, Cbar, Dbar, u[:100], torch.zeros((10,1)))

In [15]:
print((abs(conv_fft - conv_notfft)<1e-5).all())
print((abs(conv_fft - y)<1e-5).all())

tensor(True)
tensor(True)


In [16]:
def log_step_initializer(dt_min = 0.001, dt_max = 0.1):
    """
    initial guess for dt, from random number generator. to be learned.

    parameters:
        dt_min
        dt_max
    """
    return torch.autograd.Variable(torch.rand(1) * (torch.log(dt_max) - torch.log(dt_min)) + torch.log(dt_min), requires_grad = True)

In [17]:
class SSMLayer(torch.nn.Module):
    """
    Simple layer that does SSMing. Assumes single input, single output. 
    Could be made multi-dimensional either by stacking and decorrelating,
    or by playing with the code to allow for multi input, multioutput. Should be relatively easy, 
    but need to carefully think a little about convolution of multi dim inputs.
    """
    def __init__(
        self,
        latent_dim,
        L_max,
        dt_min = 0.001,
        dt_max = 0.1,
    ):
        super().__init__()
        self.latent_dim = latent_dim
        self.A, self.B, self.C, self.D = self.random_SSM(latent_dim)
        self.Abar, self.Bbar, self.Cbar, self.Dbar = self.discretize(self.A, self.B, self.C, self.D, self.dt)
        self.dt = self.log_step_initializer(dt_min, dt_max)

    def random_SSM(
        self, 
        N : int
    ) -> (torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor):
        """
        initializing SSM parameters given latent dim
        
        parameters:
            N : size of latent dimension
        """
        A = torch.autograd.Variable(torch.rand(size=(N,N)), requires_grad = True)
        B = torch.autograd.Variable(torch.rand(size=(N,1)), requires_grad = True)
        C = torch.autograd.Variable(torch.rand(size=(1,N)), requires_grad = True)
        D = torch.autograd.Variable(torch.rand(size=(1,1)), requires_grad = True)
        return A, B, C, D

    def log_step_initializer(self, dt_min = 0.001, dt_max = 0.1):
        """
        initial guess for dt, from random number generator. to be learned.
    
        parameters:
            dt_min
            dt_max
        """
        return torch.autograd.Variable(torch.rand(1) * (torch.log(dt_max) - torch.log(dt_min)) + torch.log(dt_min), requires_grad = True)

    def discretize(
        self, A : torch.Tensor, B : torch.Tensor, C : torch.Tensor, D : torch.Tensor, delta : torch.Tensor
    ) -> (torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor):
        """Discretizes SSM using bilinear model
    
        parameters:
            A: (NxN) transition matrix in latent
            B: (Nx1) projection matrix to latent
            C: (1xN) projection matrix from latent to output
            D: (1x1) skip connection from input to output
            delta: time step, ensure sufficient smallness
        """
        Cbar = C
        Dbar = D
        N = A.shape[0]
        Bl = torch.linalg.inv(torch.eye(N) - delta / 2 * A)
        Abar = Bl@(torch.eye(N) + delta/2 * A)
        Bbar = Bl@(delta*B)
        return Abar, Bbar, Cbar, Dbar

    def scan_SSM(
        self, Ab : torch.Tensor, Bb : torch.Tensor, Cb : torch.Tensor, Db : torch.Tensor,  u : torch.Tensor, x0 : torch.Tensor
    ) -> torch.Tensor:
        """
        computes steps of the SSM going forward.
    
        parameters:
            Ab : (NxN) transition matrix in discrete space of latent to latent
            Bb : (Nx1) projcetion matrix from input to latent space
            Cb : (1xN) projection matrix from latent to output
            Db : (1x1) skip connection input to output
            u  : (L,)  trajectory we are trying to track
            x0 : (Nx1) initial condition of latent
        """
        x0 = torch.zeros((10,1))
        x = torch.zeros((Ab.shape[0], len(u[:100])))
        y = torch.zeros_like(u[:100])
        for i in range(u[:100].shape[0]):
            x[:,i] = (Ab@x0 + Bb*u[i]).squeeze()
            y[i] = (Cb@x[:,i]).squeeze()
            x0 = x[:,i].unsqueeze(-1)
        return x, y
        
    def K_conv(self, Ab : torch.Tensor, Bb : torch.Tensor, Cb : torch.Tensor, L : int) -> torch.Tensor:
        """
        computes convolution window given L time steps using equation K_t = Cb @ (Ab^t) @ Bb. 
        Needs to be flipped for correct causal convolution, but can be used as is in fft mode
    
        parameters:
            Ab : transition matrix
            Bb : projection matrix from input to latent
            Cb : projection matrix from latent to input
            Db : skip connection
            L  : length over which we want convolutional window
        """
        return torch.stack([(Cb @ torch.matrix_power(Ab, l) @ Bb).squeeze() for l in range(L)])

    def causal_conv(u : torch.Tensor, K : torch.Tensor, notfft : bool = False) -> torch.Tensor:
        """
        computes 1-d causal convolution either using standard method or fft transform.
    
        parameters:
            u : trajectory to convolve
            K : convolutional filter
            notfft: boolean, for whether or not we use fft mode or not.
        """
        assert K.shape==u.shape
        
        L = u.shape[0]
        powers_of_2 = 2**int(math.ceil(math.log2(2*L)))
    
        if notfft:
            padded_u = torch.nn.functional.pad(u, (L-1,L-1))
            convolve = torch.zeros_like(u)
            for i in range(L):
                convolve[i] = torch.sum(padded_u[i:i+L]*K.flip(dims=[0]))
            return convolve
        else:
    
            K_pad = torch.nn.functional.pad(K, (0, L))
            u_pad = torch.nn.functional.pad(u, (0, L))
            
            K_f, u_f = torch.fft.rfft(K_pad, n = powers_of_2), torch.fft.rfft(u_pad, n = powers_of_2)
            return torch.fft.irfft(K_f * u_f, n = powers_of_2)[:L]

    def forward(
        self,
        u : torch.Tensor,
        x0 : torch.Tensor = torch.zeros((1,1)),
        mode : bool | str = False
    ) -> torch.Tensor:
        """
        forward pass of model

        Parameters:
            u  : input time series
            x0 : initial condition, only used in recurrent mode
            mode: recurrent mode ("recurrent"), or convolution mode (True : direct convolution, False : fourier transform)
        """
        if mode == "recurrent":
            return self.scan_SSM(self.Abar, self.Bbar, self.Cbar, u, x0)[1]
        else:
            K = self.K_conv(self.Abar, self.Bbar, self.Cbar, u.shape[0])
            return self.causal_conv(u, K, mode)

In [50]:
def make_HiPPO(N : int) -> torch.Tensor:
    """
    creates HiPPO matrix for legendre polynomials up to order N
    parameters:
        N: int
    """
    P = torch.sqrt(1+2*torch.arange(N))
    A = P.unsqueeze(1) * P.unsqueeze(0)
    A = torch.tril(A) - torch.diag(torch.arange(N))
    return -A

In [19]:
def K_gen_inverse(
    Abar : torch.Tensor, Bbar : torch.Tensor, Cbar : torch.Tensor, L : int
) -> torch.Tensor:
    """
    creates generating function for convolutional window, to be evaluated at roots of unity
    parameters:
        Abar : discretized A matrix
        Bbar : discretized B matrix
        Cbar : discretized C matrix
        L    : length of convolutional window
    """
    Abar = Abar.to(torch.complex64)
    Bbar = Bbar.to(torch.complex64)
    Cbar = Cbar.to(torch.complex64)
    
    I = torch.eye(Abar.shape[0]).to(torch.complex64)
    Al = torch.matrix_power(Abar, L)
    Ctilde = Cbar @ (I - (Al))
    return lambda z: (torch.conj(Ctilde)@(torch.linalg.inv(I-(Abar * z)))@Bbar).squeeze()

In [20]:
def conv_from_gen(gen : Callable, L : int):
    """
    returns convolution from generating function by evaluating at roots of unity

    parameters:
        gen : generating function
        L   : int
    """
    omega_L = torch.exp(-2j * torch.pi * torch.arange(L)/L)
    atRoots = torch.tensor([gen(omega) for omega in omega_L])
    return torch.fft.irfft(atRoots, L).squeeze()
    

In [21]:
def cauchy(v : torch.Tensor, omega : torch.Tensor, lambd : torch.Tensor) -> torch.Tensor:
    """
    helper function for calculating cauchy kernel in generating function

    parameters:
        v : a dot product vector, relying on DPLR representation of SSM matrices
        omega : complex poles
        lambd : diagonal values of A matrix stand-in
    """
    cauchy_dot = lambda _omega: (v/(omega-lamb)).sum()
    return cauchy_dot
    

In [22]:
def K_gen_DPLR(
    Lambda : torch.Tensor, 
    P : torch.Tensor, 
    Q : torch.Tensor, 
    B: torch.Tensor, 
    C : torch.Tensor, 
    delta : torch.Tensor, 
    L : int
)-> torch.Tensor:
    """
    computes convolution kernel from generating function using DPLR representation and
    the cauchy kernel

    Parameters:
        Lambda : diagonal part of A
        P : Nx1 matrix, rank 1 representation to A
        Q : Nx1 matrix, rank 1 representation to A
        C : 1xN matrix, projection from latent to input
        B : Nx1 matrix, projection from input to latent
    """
    Omega_L = torch.exp(-2j*torch.pi * (torch.arange(L))/L)

    aterm = (torch.conj(C), torch.conj(Q))
    bterm = (B, P)

    g = (2.0/delta) * ((1.0-Omega_L)/(1.0+Omega_L))
    c = 2.0 / (1.0+Omega_L)

    k00 = cauchy(aterm[0] * bterm[0], g, Lambda)
    k01 = cauchy(aterm[0] * bterm[1], g, Lambda)
    k10 = cauchy(aterm[1] * bterm[0], g, Lambda)
    k11 = cauchy(aterm[1] * bterm[1], g, Lambda)

    atRoots = c * (k00 - k01 * (1.0 / (1.0 + k11)) * k10)
    out = np.fft.irfft(atRoots, L)
    return out

In [23]:
def discrete_DPLR(
    Lambda : torch.Tensor,
    P : torch.Tensor,
    Q : torch.Tensor,
    B : torch.Tensor,
    C : torch.Tensor,
    delta : torch.Tensor,
    L : int
)->(torch.Tensor, torch.Tensor, torch.Tensor):
    """
    computes the discretized version of the state space model,
    assuming the DPLR form

    Parameters:
        Lambda : Nx1, represents the diagonal values of the A matrix
        P : Nx1, represents part of the low rank aspect of the A matrix
        Q : Nx1, represents the other part of the low rank aspect of the A matrix
        B : N, projection from input to latent
        C : N, projection from latent to input
        delta : step size
        L : length of window
    """
    Bt = B.unsqueeze(1)
    Ct = C.unsqueeze(0)

    A = (torch.diag(Lambda) - torch.outer(P, torch.conj(Q)))
    A0 = 2.0/delta * torch.eye(A.shape[0]) + A

    Qdagger = torch.conj(torch.transpose(Q))
    
    D = torch.diag(1.0/(2.0/delta - Lambda))
    A1 = (D -  (1.0/(1.0 + Qdagger @ D @ P)) * D@P@Qdagger@D)
    Ab = A1@A0
    Bb = 2 * A1
    Cb = Ct @ torch.conj(torch.linalg.inv(torch.eye(A.shape[0]) - torch.matrix_power(Ab, L)))
    return Ab, Bb, Cb

In [24]:
def make_NPLR_HiPPO(N : int) -> torch.Tensor:
    """
    creating hippo matrix and associated low rank additive component, P
    and the B matrix associated, as hippo forces it

    parameters:
        N : int, degree of legendre polynomial coefficient
    """
    nhippo = make_HiPPO(N)

    P = torch.sqrt(torch.arange(N)+0.5).to(torch.complex64)
    B = torch.sqrt(2*torch.arange(N)+1.0).to(torch.complex64)

    return nhippo.to(torch.complex64), P, B


In [55]:
def make_DPLR_HiPPO(N : int) -> torch.Tensor:
    """
    convert matrices to DPLR representation
    parameters:
        N : int, degree of legendre polynomials
    """
    A, P, B = make_NPLR_HiPPO(N)

    S = A + torch.outer(P, P)

    S_diag = torch.diagonal(S)
    Lambda_real = torch.mean(S_diag) * torch.ones_like(S_diag)

    Lambda_imag, V = torch.linalg.eigh(S * -1j)
    P = V.T.conj() @ P
    B = V.T.conj() @ B
    return Lambda_real + 1j * Lambda_imag, P, B, V

In [61]:
N=8
A2, P, B = make_NPLR_HiPPO(N)
Lambda, Pc, Bc, V = make_DPLR_HiPPO(N)
Vc = V.conj().T
P = P
Pc = Pc
Lambda = torch.diag(Lambda)
A3 = V @ Lambda @ Vc - torch.outer(P,P.conj())  # Test NPLR
A4 = V @ (Lambda - torch.outer(Pc,Pc.conj())) @ Vc  # Test DPLR
print(A2)
print(A3)
print(A4)
assert torch.allclose(A2, A3, atol=1e-4, rtol=1e-4)
assert torch.allclose(A2, A4, atol=1e-4, rtol=1e-4)

tensor([[ -1.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -1.7321+0.j,  -2.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -2.2361+0.j,  -3.8730+0.j,  -3.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -2.6458+0.j,  -4.5826+0.j,  -5.9161+0.j,  -4.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.0000+0.j,  -5.1962+0.j,  -6.7082+0.j,  -7.9373+0.j,  -5.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.3166+0.j,  -5.7446+0.j,  -7.4162+0.j,  -8.7750+0.j,  -9.9499+0.j,  -6.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.6056+0.j,  -6.2450+0.j,  -8.0623+0.j,  -9.5394+0.j, -10.8167+0.j, -11.9583+0.j,
          -7.0000+0.j,  -0.0000+0.j],
        [ -3.8730+0.j,  -6.7082+0.j,  -8.6603+0.j, -10.2470+0.j, -11.6189+0.j, -12.

In [62]:
A2

tensor([[ -1.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -1.7321+0.j,  -2.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -2.2361+0.j,  -3.8730+0.j,  -3.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -2.6458+0.j,  -4.5826+0.j,  -5.9161+0.j,  -4.0000+0.j,  -0.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.0000+0.j,  -5.1962+0.j,  -6.7082+0.j,  -7.9373+0.j,  -5.0000+0.j,  -0.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.3166+0.j,  -5.7446+0.j,  -7.4162+0.j,  -8.7750+0.j,  -9.9499+0.j,  -6.0000+0.j,
          -0.0000+0.j,  -0.0000+0.j],
        [ -3.6056+0.j,  -6.2450+0.j,  -8.0623+0.j,  -9.5394+0.j, -10.8167+0.j, -11.9583+0.j,
          -7.0000+0.j,  -0.0000+0.j],
        [ -3.8730+0.j,  -6.7082+0.j,  -8.6603+0.j, -10.2470+0.j, -11.6189+0.j, -12.

In [67]:
A3[-1]

tensor([ -3.8730-3.9634e-07j,  -6.7082-2.8920e-06j,  -8.6603-2.3747e-06j,
        -10.2470-1.3913e-06j, -11.6189-2.3939e-06j, -12.8452-5.5860e-07j,
        -13.9642+9.4411e-07j,  -8.0000+2.2398e-06j])

In [162]:
test_nplr()

tensor([[ 1.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 1.7321+0.j,  2.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 2.2361+0.j,  3.8730+0.j,  3.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 2.6458+0.j,  4.5826+0.j,  5.9161+0.j,  4.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 3.0000+0.j,  5.1962+0.j,  6.7082+0.j,  7.9373+0.j,  5.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 3.3166+0.j,  5.7446+0.j,  7.4162+0.j,  8.7750+0.j,  9.9499+0.j,  6.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 3.6056+0.j,  6.2450+0.j,  8.0623+0.j,  9.5394+0.j, 10.8167+0.j, 11.9583+0.j,
          7.0000+0.j,  0.0000+0.j],
        [ 3.8730+0.j,  6.7082+0.j,  8.6603+0.j, 10.2470+0.j, 11.6189+0.j, 12.8452+0.j,
         13.9642+0.j,  8.0000+0.j]])
tensor([[  8.00

AssertionError: 

In [134]:
stuff.T.conj()

tensor([[0.5818, 0.1010, 0.2738, 0.4646, 0.4686, 0.7683, 0.3008, 0.1447, 0.6443,
         0.2300],
        [0.4416, 0.3593, 0.4735, 0.5249, 0.9900, 0.6918, 0.0567, 0.2545, 0.0569,
         0.0589],
        [0.3173, 0.7228, 0.6870, 0.1064, 0.7708, 0.1748, 0.7994, 0.6445, 0.1681,
         0.1523],
        [0.0278, 0.0657, 0.8890, 0.6282, 0.3107, 0.5344, 0.8567, 0.9261, 0.9850,
         0.0713],
        [0.6945, 0.2748, 0.0964, 0.1249, 0.9149, 0.5747, 0.9080, 0.0049, 0.8492,
         0.6751],
        [0.7639, 0.7042, 0.0486, 0.5208, 0.3522, 0.8219, 0.8999, 0.0682, 0.4664,
         0.5055],
        [0.8660, 0.8240, 0.4073, 0.1050, 0.7550, 0.7808, 0.8962, 0.9453, 0.2059,
         0.7177],
        [0.7166, 0.1512, 0.7124, 0.8595, 0.1382, 0.5977, 0.2991, 0.1247, 0.4054,
         0.5614],
        [0.3957, 0.3469, 0.2057, 0.4245, 0.7076, 0.4653, 0.6787, 0.8338, 0.7559,
         0.9425],
        [0.8095, 0.1464, 0.8209, 0.0577, 0.9746, 0.0251, 0.8017, 0.7375, 0.7066,
         0.1027]])